In [174]:
import json
import numpy as np
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/all.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 47479434


In [5]:
stopwords = set(one_time)

In [164]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Первая неделя марта 2015

In [177]:
mart_start = 1425157200000
mart_end = 1425762000000

In [178]:
mart_news = list(filter(lambda x: x.date > mart_start and x.date < mart_end, news))
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 625


In [179]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)
tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 18525


## DBScan

In [180]:
from sklearn.cluster import DBSCAN

In [181]:
# X = (tfidf_matrix * tfidf_matrix.T).A

In [225]:
db = DBSCAN(eps=1.3, min_samples=3, n_jobs=-1).fit(tfidf_matrix)
# 1.36 for (1, 3)
# 1.12 for 1

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
labels[:100]
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 19
-1: 183, 0: 3


In [226]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 183, titles:
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	В России могут появиться продуктовые карточки для льготников и бедных
	Сумерки — Царского Села под Петербургом разукрасили огни светового шоу
	Американского агента Томаса выдворили из России
	Пореченков назвал слухи о ссоре с "Беларусьфильм" ложью
	Ребенок из России назван самым трогательным суперзлодеем Интернета
	В России вспоминают подвиг Псковских десантников
	Где летом в Москве будут самые длинные пробки
	Футболисты «Челси» выиграли Кубок английской Лиги
	Одного из первых террористов СССР до сих пор содержат в кубанской спецклинике
	Турфирмы зафиксировали спад по всем массовым направлениям
	Сергей Пугачев отказался от адвокатов из-за нехватки средств
	Вся Британия обсуждает ролик, на котором из окна Букингемского дворца вылезает… голый мужчина
	Участники силовой операции записали видеопослание 

# KMeans

In [185]:
from sklearn.cluster import KMeans

In [186]:
num_clusters = 50

km = KMeans(n_clusters=num_clusters, max_iter=100, n_init=10, n_jobs=-1).fit(tfidf_matrix)

clusters = km.cluster_centers_
labels = km.labels_
print_topics(km.cluster_centers_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
ветеран стипендия студент война победа великий живой азаров участник немец орден город битва страшный псковский схонберт деньги брюгге задержка фронт
Topic #1:
польша польский армия украинский бандера мэннинг немецкий военный документ изнасилование война территория красный чехия перлюстрация поляк цветок советский схетин тысяча
Topic #2:
человек лошагин мочь видео россия машина становиться самый убивать журналист убийство новый ролик хотеть насилие сноудно рассказывать несколько кирпич роскомнадзор
Topic #3:
женщина мужчина ученый исследование россия сфера татьяна операция военный процент дударев пенис окава женский размер данные камбоджа альбинос длина слово
Topic #4:
шествие немцов москворецкий борис участник акция траурный марш проходить память проезд мост китайгородский москва человек мэрия воскресение оппозиционер центр большой
Topic #5:
немцов убийство борис мост преступление дурицкая версия следствие политик москворецкий машина камера дело убивать москва анна февраль м

In [187]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 10, titles:
	В России вспоминают подвиг Псковских десантников
	Не стареют душой ветераны...
	Ветеран Виктор Азаров: лет десять после войны снилось, что попадал в плен
	Студенты более 40 вузов пожаловались на задержку стипендий
	Детей Донбасса сейчас принимают в Абхазии
	Башкирские ветераны войны помогают волонтерам создать архив воспоминаний
	Смертельно больной фанат отложил эвтаназию ради посещения матча
	Великий Новгород был практически уничтожен во время Великой Отечественной войны
	Жизнь взаймы: российские вузы погасят долги по стипендиям до конца недели
	В Иркутске расследуют убийство ветерана Великой Отечественной и его супруги
Cluster: 1, count news: 13, titles:
	Пока Красная Армия освобождала Польшу, местное подполье стреляло ей в спину
	Журналистку оштрафовали на тысячу рублей за фото Смоленска
	«У поляков такое же имперское мышление, что и у русских»
	Россия отказалась пустить спикера сената Польши на похороны Немцова
	В отношениях Киева с Варшавой мож

# KMeans with LSA

# LSA

In [188]:
from sklearn.decomposition import TruncatedSVD

In [189]:
lsa = TruncatedSVD(n_components=50, n_iter=100)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [190]:
print_topics(lsa.components_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
немцов убийство борис дурицкая мост москва анна россия человек дело преступление мочь москворецкий политик украинский украина следствие политика февраль центр
Topic #1:
украина россия российский президент компания украинский цена губернатор хорошавин шахта страна рубль помощь правительство савченко санкция взрыв проект человек становиться
Topic #2:
шахта взрыв засядько горняк донецк украина украинский помощь гуманитарный шахтер погибший пострадать колонна донецкий савченко авария погибать семья метан спасатель
Topic #3:
губернатор хорошавин сахалинский область сахалин александр президент глава задерживать савченко задержание следственный взятка обыск подозреваемый регион правительство дадаев дело арест
Topic #4:
савченко украинский украина российский надежда порошенко голодовка летчица президент гончаренко сизо письмо путин депутат решение заявлять граница журналист вгтрк пересечение
Topic #5:
цена нефть доллар рубль курс шахта торги биржа компания баррель россия рынок немцов

In [191]:
lsa_labels = [x.argmax() for x in lsa_matrix]
print_clusters(mart_news, lsa_labels)

Cluster: 0, count news: 131, titles:
	Смерть Немцова: «Минута славы» для бывших
	Из-за траурного марша движение в центре Москвы ограничат
	До поздней ночи в Москве несли цветы к месту гибели Бориса Немцова
	Гражданке Украины Дурицкой посоветовали оставаться в Москве
	Крупнейшие бизнесмены не пойдут на шествие памяти Бориса Немцова
	В центре Москвы завершился марш памяти Бориса Немцова
	Марш в память о Немцове: в Москве усилена безопасность
	В центре Москвы начался марш памяти Бориса Немцова
	Свидетель описал убийцу Бориса Немцова, составлен портрет
	За информацию об убийстве Немцова дадут три миллиона
	Траурный марш памяти Бориса Немцова: онлайн-трансляция
	Немцову хотят поставить памятник в Москве
	Убийство Немцова расследует специалист по делам националистов
	Убийство Немцова: полиция нашла владельца разыскиваемой "десятки"
	Спутница Немцова отказалась от программы защиты свидетелей
	Киселеву будет не хватать Немцова
	Кабинет Немцова обыскивают
	Дурицкая потеряла память после расстре

# LDA

In [192]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, tokenizer=lambda text: text.split(" "), stop_words=stopwords)
tf = tf_vectorizer.fit_transform(mart_content)

In [218]:
lda = LatentDirichletAllocation(n_topics=100, max_iter=100, learning_method='online',
                                learning_offset=50., )
lda_matrix = lda.fit_transform(tf)
print_topics(lda.components_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
невыездной невыполнение лукашевич второй комплектоваться ляхов киров википедия лукин возликовать бытовуха минимальный логика координатор дефендеры дошкольный бригада акунин заморочка невыносимый
Topic #1:
анатолий антифашистский неожиданный дистанционно монреаль германия воспроизводить доктор глинка кряжев знакомиться маттео антонина куприянов дебальцевский анатолийский детализация высококлассный возможно багаж
Topic #2:
ляхов коммуналка африка мечтать климентьев инстанция обладать контур минфин комментироваться легко задержание манчестер доверие дошкольный комплектоваться лида камбоджа заново ангельский
Topic #3:
будни катынь верить домодедово автосалон невыездной биллинг лукин конкурентоспособность запылиться дома запутываться икрамов минимальный жмурки второй бывший задание многое изобличение
Topic #4:
логика выборка диктатура веранда благоговение локоть допрашиваться гибрид неустанный неспособный невьянск купальник маникюрщица квалифицировать движущий выявлять мотя антроп

In [220]:
lda_labels = [x.argmax() for x in lda_matrix]
print_clusters(mart_news, lda_labels)

Cluster: 0, count news: 10, titles:
	Глава СПЧ не уговорил Савченко начать есть
	Порошенко присвоил Надежде Савченко звание Героя Украины
	Надежда Савченко пообещала своему адвокату прекратить голодовку
	Адвокат: Савченко откажется от голодовки, "когда станет совсем ужасно"
	Джохару Царнаеву подобрали присяжных
	Кремль ответит на письмо Порошенко о Савченко
	Савченко попросила предоставить ей сбалансированное питание
	Порошенко рассказал о содержании письма Путина про Савченко
	Мать Савченко обратилась за помощью к Меркель
	Киев рассказал, что было в письме Путина Порошенко
Cluster: 6, count news: 405, titles:
	Американский астронавт почтил память Леонарда Нимоя в космосе
	Власти Севастополя национализировали завод Порошенко
	Что изменилось с 1 марта: Кредиты - в хорошие руки, лекарства - под госконтроль
	Лондон заблокировал сделку по покупке фондом Фридмана «дочки» RWE
	ЦБ назвал «немного чрезмерным» прогноз Moody's по инфляции в 22%
	С 1 марта россияне смогут въехать на Украину тольк